<a href="https://colab.research.google.com/github/dupeljan/collab/blob/main/Copy_of_writing_a_training_loop_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Writing a training loop from scratch

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

Setup dataset

In [11]:
# Prepare the training dataset.
batch_size = 64
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = np.reshape(x_train, (-1, 784))
x_test = np.reshape(x_test, (-1, 784))

# Reserve 10,000 samples for validation.
x_val = x_train[-10000:]
y_val = y_train[-10000:]
x_train = x_train[:-10000]
y_train = y_train[:-10000]

# Prepare the training dataset.
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

# Prepare the validation dataset.
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.batch(batch_size)

# Eager mode

In [12]:
inputs = keras.Input(shape=(784,), name="digits")
x1 = layers.Dense(64, activation="relu")(inputs)
x2 = layers.Dense(64, activation="relu")(x1)
outputs = layers.Dense(10, name="predictions")(x2)
model = keras.Model(inputs=inputs, outputs=outputs)

In [13]:
# Instantiate an optimizer.
optimizer = keras.optimizers.SGD(learning_rate=1e-3)
# Instantiate a loss function.
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)



In [14]:
def train(model, epochs = 2):
  for epoch in range(epochs):
      print("\nStart of epoch %d" % (epoch,))

      # Iterate over the batches of the dataset.
      for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):

          # Open a GradientTape to record the operations run
          # during the forward pass, which enables auto-differentiation.
          with tf.GradientTape() as tape:

              # Run the forward pass of the layer.
              # The operations that the layer applies
              # to its inputs are going to be recorded
              # on the GradientTape.
              logits = model(x_batch_train, training=True)  # Logits for this minibatch

              # Compute the loss value for this minibatch.
              loss_value = loss_fn(y_batch_train, logits)

          # Use the gradient tape to automatically retrieve
          # the gradients of the trainable variables with respect to the loss.
          grads = tape.gradient(loss_value, model.trainable_weights)

          # Run one step of gradient descent by updating
          # the value of the variables to minimize the loss.
          optimizer.apply_gradients(zip(grads, model.trainable_weights))

          # Log every 200 batches.
          if step % 200 == 0:
              print(
                  "Training loss (for one batch) at step %d: %.4f"
                  % (step, float(loss_value))
              )
              print("Seen so far: %s samples" % ((step + 1) * 64))
train(model)


Start of epoch 0
Training loss (for one batch) at step 0: 84.5313
Seen so far: 64 samples
Training loss (for one batch) at step 200: 2.1334
Seen so far: 12864 samples
Training loss (for one batch) at step 400: 0.7563
Seen so far: 25664 samples
Training loss (for one batch) at step 600: 0.6857
Seen so far: 38464 samples

Start of epoch 1
Training loss (for one batch) at step 0: 0.6501
Seen so far: 64 samples
Training loss (for one batch) at step 200: 0.5389
Seen so far: 12864 samples
Training loss (for one batch) at step 400: 0.5519
Seen so far: 25664 samples
Training loss (for one batch) at step 600: 0.5096
Seen so far: 38464 samples


Check trainable weights

In [15]:
[ x.name for x in model.trainable_weights]

['dense_2/kernel:0',
 'dense_2/bias:0',
 'dense_3/kernel:0',
 'dense_3/bias:0',
 'predictions/kernel:0',
 'predictions/bias:0']

Freeze layers

In [16]:
def copy(weights):
    res = []
    for w in weights:
      res.append(tf.identity(w))
    return res 


weights = dict()
# Freeze all layers besides the last one
target_layer_name = 'predictions'
for layer in model.layers:
  if layer.weights:
    if layer.name != target_layer_name:
      layer.trainable = False
    weights[layer.name] = copy(layer.weights)

# Check trainable weights list
[x.name for x in model.trainable_weights]

['predictions/kernel:0', 'predictions/bias:0']

Train only last layer of model and check it


In [17]:
train(model)

def equal(w1, w2):
    return [ bool(tf.math.reduce_all(tf.math.equal(w, w_saved))) 
              for w, w_saved in zip(w1, w2)]

    
for layer in model.layers:
  if not layer.weights:
    continue
  if layer.name != target_layer_name:
    assert all(equal(weights[layer.name], layer.weights))
  else:
    assert not all(equal(weights[layer.name], layer.weights))
  print("{0} sqr diff: {1}".format(
        layer.name,
        tf.math.reduce_sum(tf.math.pow(
                        layer.weights[0] -  weights[layer.name][0], 2))
    ))



Start of epoch 0
Training loss (for one batch) at step 0: 0.3266
Seen so far: 64 samples
Training loss (for one batch) at step 200: 0.3251
Seen so far: 12864 samples
Training loss (for one batch) at step 400: 0.6280
Seen so far: 25664 samples
Training loss (for one batch) at step 600: 0.2726
Seen so far: 38464 samples

Start of epoch 1
Training loss (for one batch) at step 0: 0.2719
Seen so far: 64 samples
Training loss (for one batch) at step 200: 0.7506
Seen so far: 12864 samples
Training loss (for one batch) at step 400: 0.3221
Seen so far: 25664 samples
Training loss (for one batch) at step 600: 0.4017
Seen so far: 38464 samples
dense_2 sqr diff: 0.0
dense_3 sqr diff: 0.0
predictions sqr diff: 0.02466883882880211


Freezing in eager mode is working!

You can freeze layer, but you can't set weight trainable attribute. If layer have trainable == false this it just not lised at `model.trainable_weights`

In [ ]:
for layer in model.layers:
  if layer.name != target_layer_name:
    for weight in layer.weights:
      print('w', weight.name, weight.trainable)
    print('l', layer.name, layer.trainable)

l digits False
w dense/kernel:0 True
w dense/bias:0 True
l dense False
w dense_1/kernel:0 True
w dense_1/bias:0 True
l dense_1 False


# Non eager mode with GradTape

Setup train loop

In [18]:
@tf.function
def train():
  epochs = 3
  for epoch in range(epochs):
      print("\nStart of epoch %d" % (epoch,))

      # Iterate over the batches of the dataset.
      for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):

          # Open a GradientTape to record the operations run
          # during the forward pass, which enables auto-differentiation.
          with tf.GradientTape() as tape:

              # Run the forward pass of the layer.
              # The operations that the layer applies
              # to its inputs are going to be recorded
              # on the GradientTape.
              logits = model(x_batch_train, training=True)  # Logits for this minibatch

              # Compute the loss value for this minibatch.
              loss_value = loss_fn(y_batch_train, logits)

          # Use the gradient tape to automatically retrieve
          # the gradients of the trainable variables with respect to the loss.
          grads = tape.gradient(loss_value, model.trainable_weights)

          # Run one step of gradient descent by updating
          # the value of the variables to minimize the loss.
          optimizer.apply_gradients(zip(grads, model.trainable_weights))



Create model

In [19]:
inputs = keras.Input(shape=(784,), name="digits")
x1 = layers.Dense(64, activation="relu")(inputs)
x2 = layers.Dense(64, activation="relu")(x1)
outputs = layers.Dense(10, name="predictions")(x2)
model = keras.Model(inputs=inputs, outputs=outputs)

Setup trainable params

In [20]:
def copy(weights):
    res = []
    for w in weights:
      res.append(tf.identity(w))
    return res 


weights = dict()
def save_weight():
  global weights
  for layer in model.layers:
    if layer.weights:
      weights[layer.name] = copy(layer.weights)

save_weight()

# Freeze all layers besides the last one
target_layer_name = 'predictions'
for layer in model.layers:
  if layer.weights:
    if layer.name != target_layer_name:
      layer.trainable = False

# Check trainable weights list
[x.name for x in model.trainable_weights]

['predictions/kernel:0', 'predictions/bias:0']

Try to train only last layer

In [21]:
train()


Start of epoch 0

Start of epoch 1

Start of epoch 2


Check freezing

In [22]:

def equal(w1, w2):
    return [ bool(tf.math.reduce_all(tf.math.equal(w, w_saved))) 
              for w, w_saved in zip(w1, w2)]

def check_is_trainable():    
  for layer in model.layers:
    if not layer.weights:
      continue
    if layer.name != target_layer_name:
      assert all(equal(weights[layer.name], layer.weights))
    else:
      assert not all(equal(weights[layer.name], layer.weights))
    print("{0} sqr diff: {1}".format(
          layer.name,
          tf.math.reduce_sum(tf.math.pow(
                          layer.weights[0] -  weights[layer.name][0], 2))
      ))

check_is_trainable()

dense_4 sqr diff: 0.0
dense_5 sqr diff: 0.0
predictions sqr diff: 13.99891471862793


That's ok

Try to unfreeze all layers

In [23]:
for layer in model.layers:
  layer.trainable = True

In [24]:
save_weight()
train()
check_is_trainable()

dense_4 sqr diff: 0.0
dense_5 sqr diff: 0.0
predictions sqr diff: 0.32455798983573914


No success

# Keras API training mode

Create model with custom train_step to be sure that trainable vars is got from self.trainable_variables

In [15]:
class CustomModel(keras.Model):
    def train_step(self, data):
        # Unpack the data. Its structure depends on your model and
        # on what you pass to `fit()`.
        x, y = data

        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)  # Forward pass
            # Compute the loss value
            # (the loss function is configured in `compile()`)
            loss = self.compiled_loss(y, y_pred, regularization_losses=self.losses)

        # Compute gradients
        trainable_vars = self.trainable_variables
        print("Trainable vars", trainable_vars)
        gradients = tape.gradient(loss, trainable_vars)
        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        # Update metrics (includes the metric that tracks the loss)
        self.compiled_metrics.update_state(y, y_pred)
        # Return a dict mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}

In [16]:
inputs = keras.Input(shape=(784,), name="inputs")
x1 = layers.Dense(64, activation="relu", name="dense1")(inputs)
x2 = layers.Dense(64, activation="relu", name="dense2")(x1)
outputs = layers.Dense(10, name="predictions")(x2)
model = CustomModel(inputs=inputs, outputs=outputs)
[x.trainable for x in model.layers]

[True, True, True, True]

Make callable which can freeze learning








In [12]:
class FreezingCallback(tf.keras.callbacks.Callback):
  @staticmethod 
  def copy(weights):
    res = []
    for w in weights:
      res.append(tf.identity(w))
    return res 

  @staticmethod
  def have_weights(layer):
    return hasattr(layer, 'weights') and layer.weights

  @staticmethod
  def equal(w1, w2):
    return [(w.name, bool(tf.math.reduce_all(tf.math.equal(w, w_saved)))) 
              for w, w_saved in zip(w1, w2)]
    
  def __init__(self, layers_to_freeze, epoch_to_freeze):
    super().__init__()
    self.layers_to_freeze = layers_to_freeze
    self.epoch_to_freeze = epoch_to_freeze
    self.weights = dict()

  def on_epoch_begin(self, epoch, logs=None):
    # Keep weights before epoch
    for layer in self.model.layers:
      if self.have_weights(layer):
        self.weights[layer.name] = self.copy(layer.weights)
        assert all([x[1] for x in 
                   self.equal(layer.weights, self.weights[layer.name])])
        
    # Freeze if it's time to freeze
    if epoch == self.epoch_to_freeze:
      for layer in self.model.layers:
        if layer.name in self.layers_to_freeze:
          print(f"Freeze layer {layer.name}")
          layer.trainable = False

    print("Trainable status:")
    for layer in self.model.layers:
      print(f"Layer {layer.name} trainable = {layer.trainable}")
  
  def on_epoch_end(self, epoch, logs=None):
    print(logs)
    for layer in self.model.layers:
      if self.have_weights(layer):
        equal = self.equal(layer.weights, self.weights[layer.name])
        print(f"Layer {layer.name} changed? "
              f"{[(x[0], not x[1]) for x in equal]}")
        print("sqr diff {}".format(
            tf.math.reduce_sum(tf.math.pow(
                        layer.weights[0] - self.weights[layer.name][0], 2))))
        # Keep new weights
        self.weights[layer.name] = self.copy(layer.weights)


freezing_callback = FreezingCallback(layers_to_freeze=['dense1', 'dense2', ],#, 'predictions'],
                                      epoch_to_freeze=4)

In [32]:
# You can change trainable status only before compileing
for layer in model.layers:
  layer.trainable = True
[x.trainable for x in model.layers]

[True, True, True, True]

In [17]:
compile_kwargs = {
    'optimizer': keras.optimizers.SGD(learning_rate=1e-3), 
    'loss': keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    'metrics': [keras.metrics.SparseCategoricalAccuracy()],
}

model.compile(
    **compile_kwargs
)


In [18]:
# Setup trainable param
#for layer in model.layers:
#  layer.trainable = False
#model.trainable = False

model.fit(
    x_train,
    y_train,
    batch_size=32,
    epochs=10,
    callbacks=[freezing_callback]
)


Epoch 1/10
Trainable status:
Layer inputs trainable = True
Layer dense1 trainable = True
Layer dense2 trainable = True
Layer predictions trainable = True
Trainable vars [<tf.Variable 'dense1/kernel:0' shape=(784, 64) dtype=float32>, <tf.Variable 'dense1/bias:0' shape=(64,) dtype=float32>, <tf.Variable 'dense2/kernel:0' shape=(64, 64) dtype=float32>, <tf.Variable 'dense2/bias:0' shape=(64,) dtype=float32>, <tf.Variable 'predictions/kernel:0' shape=(64, 10) dtype=float32>, <tf.Variable 'predictions/bias:0' shape=(10,) dtype=float32>]
Trainable vars [<tf.Variable 'dense1/kernel:0' shape=(784, 64) dtype=float32>, <tf.Variable 'dense1/bias:0' shape=(64,) dtype=float32>, <tf.Variable 'dense2/kernel:0' shape=(64, 64) dtype=float32>, <tf.Variable 'dense2/bias:0' shape=(64,) dtype=float32>, <tf.Variable 'predictions/kernel:0' shape=(64, 10) dtype=float32>, <tf.Variable 'predictions/bias:0' shape=(10,) dtype=float32>]
1563/1563 [==============================] - 2s 1ms/step - loss: 4.1225 - spar